In [218]:
# Imports

import car
from auto.capabilities import acquire, release, list_caps
from car.motors import set_throttle, set_steering
from auto.camera import global_camera, close_global_camera
from auto import console
from typing import Tuple
import time

import numpy as np
import cv2
import os

list_caps()

('AHRS',
 'Accelerometer',
 'Buzzer',
 'Calibrator',
 'Camera',
 'CarMotors',
 'Credentials',
 'Encoders',
 'Gyroscope',
 'Gyroscope_accum',
 'LEDs',
 'LoopFrequency',
 'PID_steering',
 'PWMs',
 'Photoresistor',
 'Power',
 'PushButtons',
 'VersionInfo')

In [219]:
# Computer Vision constants
cv2.setNumThreads(0)

# BLUE = np.array([112,121, 189])

# As HSV (in OpenCV everything is scaled by 0.5 for some reason)
green_hue = 180 / 2 # Green is the left
blue_hue = 214 / 2 # Blue is the right
red_hue = 340 / 2
white_hue = 225 / 2
purple_hue = 275 / 2

car_hue = -1

TRACK_DIFF = 5
WHITE_DIFF = 25 # For some reason this appears to be way higher
CAR_DIFF = 10

CROP_HEIGHT = 70
CROP_LEFT = 60
CROP_RIGHT = 60

In [220]:
# Functions

def track_bounds_crop(img):
    """
    Camera captures 240 pixels of height and 320 of width
    Shape is (240, 320, 3)
    """
    return img[-CROP_HEIGHT:, CROP_LEFT:-CROP_RIGHT]


def track_zones_crop(img):
    """
    Cut out the top half
    """
    return img[-100:, :]


def scale(img, scale_percent):
    # Calculate the new dimensions
    width = int(img.shape[1] * scale_percent)
    height = int(img.shape[0] * scale_percent)
    new_size = (width, height)

    # Resize the image:
    img = cv2.resize(img, new_size, None, None, None, cv2.INTER_AREA)
    return img


def percent_color(img, target_hue, diff, scale_percent=1):
    """
    Returns the percentage of the image that is the color using HSV
    """
    hsv_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, _, _ = cv2.split(hsv_image)

    # Circular hue distance
    dist = np.abs(h - target_hue)
    dist = np.minimum(dist, 180 - dist)   # wrap-around fix

    # Pixels within threshold
    mask = (dist < diff).astype(np.uint8) * 255

    ratio = cv2.countNonZero(mask)/(img.size/3)
    color_percent = (ratio * 100) / scale_percent 
    return color_percent / 100


def throttle_to_feet_per_second(throttle):
    return 0.33514 * throttle - 3.39442


def feet_per_second_to_throttle(feet_per_second):
    return (feet_per_second + 3.39442) / 0.33541


def adjust_radius(inner_radius: float, r1: float, r2: float, angle: float) -> float:
    """
    Adjust the radius of the path taken to end up r2 away from the inner radius when starting r1 away from the inner radius
    r1 is the radius adjustment entering the turn (so average(inner_radius, outer_radius) - inner_radius would be r1 when the car starts in the middle)
    r2 is the radius adjustment exiting the turn
    angle is in radians
    """
    return np.sqrt((inner_radius + r1) * (inner_radius + r2) + (r1 - r2)**2 / (2 * np.sin(angle / 2))**2)


def too_close(frame, threshold):
    """
    If a car ahead is too close, then it is the duty of the car behind to slow down
    """
    width = frame.shape[1]

    middle_width = frame.shape[1] / 3

    l_bound = int(width - (middle_width / 2))
    r_bound = int(width + (middle_width / 2))

    if percent_color(frame[:, l_bound:r_bound], car_hue, CAR_DIFF) > threshold:
        return True
    else:
        return False


def can_overtake(frame, threshold) -> bool:
    """
    Check if there is a car that can be overtaken
    """
    width = frame.shape[1]

    middle_width = frame.shape[1] / 5

    l_bound = int(width - (middle_width / 2))
    r_bound = int(width + (middle_width / 2))

    if percent_color(frame[:, l_bound:r_bound], car_hue, CAR_DIFF) > threshold:
        return True
    else:
        return False
    

def identify_opposing_cars(frame, threshold) -> Tuple[bool, bool, bool]:
    """
    Returns three values:
    1. Is there a car to the left of the path you want to take
    2. Is there a car directly on the path you want to take
    3. Is there a car to the right of the path you want to take
    """

    # Define the bounds of "directly ahead"
    width = frame.shape[1]

    middle_width = frame.shape[1] / 10

    l_bound = int(width - (middle_width / 2))
    r_bound = int(width + (middle_width / 2))

    left_zone   = frame[:, :l_bound]
    middle_zone = frame[: l_bound:r_bound]
    right_zone  = frame[: r_bound:]

    left_percent   = percent_color(left_zone, car_hue, CAR_DIFF)
    middle_percent = percent_color(middle_zone, car_hue, CAR_DIFF)
    right_percent  = percent_color(right_zone, car_hue, CAR_DIFF)

    return (left_percent > threshold, middle_percent > threshold, right_percent > threshold)

In [221]:
# console.clear()
# while True:
#     frame = car.capture()
#     img = track_zones_crop(frame)
#     hsv_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
#     h, s, v = cv2.split(hsv_image)

#     # Circular hue distance
#     dist = np.abs(h - blue_hue)
#     dist = np.minimum(dist, 180 - dist)   # wrap-around fix

#     # Pixels within threshold
#     mask = (dist < TRACK_DIFF).astype(np.uint8) * 255
#     ratio = cv2.countNonZero(mask)/(img.size/3)
#     color_percent = (ratio * 100)
#     console.print(color_percent)

In [222]:
# Racing variables

# It is more consistent to follow the outer radius, but the inner radius can be useful in inside lines etc
inner_radii = [2.352288712045608, 1.206350190233798, 1.6627305093830878, 1.0541789388712448, 0.47572180000000003, 0.5867929778501835, 0.8351655163157062, 1.027879788271899, 1.5126722175388703, 1.634663057576926, 2.3622048]
outer_radii = [5.63312871, 4.48719019, 4.94357051, 3.51480894, 2.4442258, 3.86763298, 4.11600552, 3.48850979, 4.46542822, 4.75146106, 5.6430448]
angles = np.array([196, -90, 74, -66, 180, -114, 90, -100, -57, 67, 180])
color_order = ["RED", "NONE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "WHITE"] # Fill with the order in which the colors come for the zones, a NONE comes after a red zone without a purple immediately after

NUM_TURNS = 11
NUM_LAPS = 5
MAX_THROTTLE = 22
MIN_THROTTLE = 22

STATIC_FRICTION = 0.7
GRAVITY = 32.2 # In feet per second per second
WHEEL_BASE = 6 / 12 # 6 inches

In [223]:
# Custom logic
class Options:

    def __init__(self,
        straight_steer_bias,
        angle_threshold,
        car_threshold,
        left_threshold,
        right_threshold,
        red_brake_color_threshold,
        straight_color_threshold,
        purple_brake_color_threshold,
        acceleration_rate,
        deceleration_rate
    ):
        self.straight_steer_bias = straight_steer_bias
        self.angle_threshold = angle_threshold
        self.car_threshold = car_threshold
        self.left_threshold = left_threshold
        self.right_threshold = right_threshold
        self.red_brake_color_threshold = red_brake_color_threshold
        self.straight_color_threshold = straight_color_threshold
        self.purple_brake_color_threshold = purple_brake_color_threshold
        self.acceleration_rate = acceleration_rate
        self.deceleration_rate = deceleration_rate


class LoopInputs:
    def __init__(self, too_close: bool, can_overtake: bool, cars: Tuple[bool, bool, bool], state: str, turn_index: int, lap_number: int, angle_accum: float, bound: str):
        self.too_close = too_close
        self.can_overtake = can_overtake
        self.cars = cars
        self.state = state
        self.turn_index = turn_index
        self.lap_number = lap_number
        self.angle_accum = angle_accum
        self.bound = bound


class Car:
    def __init__(self, options: Options):
        self.instruction = {} # Any arbitrary instructions the user wants can be thrown in here, like {"action": "strafe", "start": 2 seconds ago, "duration": 5 seconds, "angle": -5}

        self.state = "STRAIGHT"
        self.next_color = "RED" # Either "RED", "WHITE", "PURPLE", or "NONE"
        self.throttle = MIN_THROTTLE
        self.angle_accum = 0
        self.lap_number = 0
        self.color_index = 0
        self.turn_index = 0

        self.acceleration_rate = options.acceleration_rate
        self.deceleration_rate = options.deceleration_rate

        # Per car
        self.straight_steer_bias = options.straight_steer_bias

        # Various thresholds when there is a continuous value to actually do something
        self.angle_threshold = options.angle_threshold
        self.car_threshold = options.car_threshold
        self.left_threshold = options.left_threshold
        self.right_threshold = options.right_threshold
        self.red_brake_color_threshold = options.red_brake_color_threshold
        self.straight_color_threshold = options.straight_color_threshold
        self.purple_brake_color_threshold = options.purple_brake_color_threshold

        # Hardware
        self.gyroscope = acquire('Gyroscope_accum')
        self.gyroscope.read()

        camera = global_camera(False)
        self.stream = camera.stream()
        next(self.stream)
    
    def fetch_inputs(self):
        """
        Returns all of the inputs needed to control the car
        """
        if self.stream:
            frame = next(self.stream)
        else:
            return None
        
        # Check for instructions to change state
        bottom_portion_img = track_zones_crop(frame)
        if self.next_color == "RED":
            # Look for red braking zones
            red_percent = percent_color(bottom_portion_img, red_hue, TRACK_DIFF)
            if red_percent > self.red_brake_color_threshold:
                self.state = "BRAKE"
                self.color_index += 1
                self.color_index %= len(color_order)
                self.next_color = color_order[self.color_index]
        elif self.next_color == "NONE":
            # Wait for there to be no more red
            red_percent = percent_color(bottom_portion_img, red_hue, TRACK_DIFF)
            if red_percent < self.red_brake_color_threshold:
                self.state = "TURN"
                self.color_index += 1
                self.color_index %= len(color_order)
                self.next_color = color_order[self.color_index]
                self.angle_accum += angles[self.turn_index]
                self.turn_index += 1
        elif self.next_color == "WHITE":
            # Look for white braking zones
            white_percent = percent_color(bottom_portion_img, white_hue, TRACK_DIFF)
            if white_percent > self.straight_color_threshold:
                self.state = "STRAIGHT"
                self.color_index += 1
                self.color_index %= len(color_order)
                self.next_color = color_order[self.color_index]
        else:
            # Look for purple braking zones, these will always come after red and be turns
            purple_percent = percent_color(bottom_portion_img, purple_hue, TRACK_DIFF)
            if purple_percent > self.purple_brake_color_threshold:
                self.state = "TURN"
                self.color_index += 1
                self.color_index %= len(color_order)
                self.next_color = color_order[self.color_index]
                self.angle_accum += angles[self.turn_index]
                self.turn_index += 1

        track_bounds_img = track_bounds_crop(frame)
        if percent_color(track_bounds_img, green_hue, TRACK_DIFF) > self.left_threshold:
            bound = "LEFT"
        elif percent_color(track_bounds_img, blue_hue, TRACK_DIFF) > self.right_threshold:
            bound = "RIGHT"
        else:
            bound = "NONE"
        
        return LoopInputs(too_close(frame, self.car_threshold), can_overtake(frame, self.car_threshold), identify_opposing_cars(frame, self.car_threshold), self.state, self.turn_index, self.lap_number, self.angle_accum, bound)


    def loop(self, inputs: LoopInputs) -> Tuple[float, float]:
        """
        User determines the throttle and steering
        """
        true_max = MAX_THROTTLE
        steer_direction = 1

        if inputs.state == "STRAIGHT":
            # Check if a car can be overtaken
            if inputs.can_overtake and not self.instruction.get("action") == "overtake":
                # On the long straights, the left line is the inside line, so we want to overtake on the left if possible
                self.instruction = {"action": "overtake", "start": time.time(), "direction": "left" if inputs.bound != "LEFT" else "right"}

            steer_bias = self.straight_steer_bias

            angle_offset = 0

            if self.instruction.get("action") == "overtake":
                # If going for more than 10 seconds, stop the attempt because something has gone wrong
                if time.time() - self.instruction.get("start") > 10:
                    self.instruction = {}
                # If the car has ended up on the edge, stop the attempt and add a late braking move to have it stick
                elif inputs.bound == "LEFT" and self.instruction.get("direction") == "left":
                    self.instruction = {"action": "brake", "start": None, "duration": 0.5}
                elif inputs.bound == "RIGHT" and self.instruction.get("direction") == "right":
                    self.instruction = {"action": "brake", "start": None, "duration": 0.5}
                else:
                    if self.instruction.get("direction") == "left":
                        angle_offset = -8
                    else:
                        angle_offset = 8
            
            z = self.gyroscope.read()[2]
            if z - (inputs.angle_accum + angle_offset) > 0:
                steer_direction = -1
            
            if z > self.angle_threshold:
                steer = 4 # Turn at a very slow, constant rate to adjust turning issues
            else:
                steer = 0

            # To mimic DRS and make racing more interesting, if there is a car directly ahead, MAX_THROTTLE is higher
            if inputs.cars[1]:
                true_max += 5

            self.throttle = true_max # Try to go flat out on straights

        elif inputs.state == "BRAKE":
            # Cancel any overtake instructions
            if self.instruction.get("action") == "overtake":
                self.instruction = {}

            steer_bias = self.straight_steer_bias
            
            z = self.gyroscope.read()[2]
            if z - inputs.angle_accum > 0:
                steer_direction = -1
            
            if z > self.angle_threshold:
                steer = 4 # Turn at a very slow, constant rate to adjust turning issues
            else:
                steer = 0

            if self.instruction.get("action") == "brake":
                if self.instruction.get("start") is not None:
                    if time.time() - self.instruction.get("start") > self.instruction.get("duration"):
                        self.throttle = 0 # This will get clipped
                        self.instruction = {}
                else:
                    self.instruction["start"] = time.time()
        else:
            """
            Due to error and good racing doesn't follow the perfect line, this can be deviated from, but anything widely
            off of this angle will be very very inefficient and maybe even dangerous
            """
            steer_bias = 0
            throttle_bias = 0

            inner_radius = inner_radii[self.turn_index]
            outer_radius = outer_radii[self.turn_index]
            r1 = (inner_radius + outer_radius) / 2 - inner_radius
            if inputs.bound == "LEFT":
                r1 = 0
            elif inputs.bound == "RIGHT":
                r1 = outer_radius - inner_radius
            r2 = (inner_radius + outer_radius) / 2 - inner_radius
            turn_radius = adjust_radius(inner_radius, r1, r2, angles[self.turn_index])
            
            steer_direction = 1 if angles[self.turn_index] > 0 else -1

            if steer_direction == 1 and inputs.bound == "LEFT" or steer_direction == -1 and inputs.bound == "RIGHT":
                # If we are on the outer radius and reach the outer line, we can just shrink our line
                turn_radius *= 0.5
            else:
                # If we have encountered the inner radius, we can't just increase the radius because we will keep driving into the circle
                # We have to reverse the direction we are turning until we no longer see the inner radius     
                steer_direction = -steer_direction

            # Convert to deg
            steer = np.arctan(WHEEL_BASE / turn_radius) * 180 / np.pi
            steer = 1.0 * steer # Account for the steering bias, different for each car
            self.throttle = feet_per_second_to_throttle(np.sqrt(turn_radius * STATIC_FRICTION * GRAVITY))
            self.throttle += throttle_bias
            
        if inputs.too_close:
            true_max = 30
        self.throttle = min(max(MIN_THROTTLE, self.throttle), true_max)
        
        return steer * steer_direction + steer_bias, self.throttle

In [224]:
console.clear()

ANGLE_THRESHOLD = 10.0 # If greater than this on a straight and approaching a boundary reduce the throttle

RED_BRAKE_COLOR_THRESHOLD = 0.8 # As a percent
STRAIGHT_COLOR_THRESHOLD = 0.6 # As a percent
PURPLE_BRAKE_COLOR_THRESHOLD = 0.6
CAR_COLOR_THRESHOLD = float('inf') # As a percent
LEFT_COLOR_THRESHOLD = 0.2 # As a percent
RIGHT_COLOR_THRESHOLD = 0.1 # As a percent

STRAIGHT_STEER_BIAS = -6.0

options = Options(STRAIGHT_STEER_BIAS, ANGLE_THRESHOLD, CAR_COLOR_THRESHOLD, LEFT_COLOR_THRESHOLD, RIGHT_COLOR_THRESHOLD, RED_BRAKE_COLOR_THRESHOLD, STRAIGHT_COLOR_THRESHOLD, PURPLE_BRAKE_COLOR_THRESHOLD, 5.0, -5.0)

car = Car(options)

throttle = MAX_THROTTLE

angle_accum = 0
lap = 0
while lap <= NUM_LAPS:
    d = car.fetch_inputs() # Obtain the track state, bounds, cars ahead, lap number, and turn index
    if d is None:
        pass
    else:
        steer, throttle = car.loop(d)
        with open("save.txt", "a+") as f:
            f.write(str(steer) + ' ' + d.state + ' ' + d.bound + ' ' + str(d.turn_index) + "\n")
        if lap == NUM_LAPS:
            set_throttle(int(throttle * 0.4))  # convert to native int
        else:
            set_throttle(int(throttle))  # convert to native int
        set_steering(steer)

release(car.gyroscope)
close_global_camera(False)

KeyboardInterrupt: 